In [42]:
import os
import errno

import numpy as np

import deepcell
from deepcell.utils.data_utils import get_data
from tensorflow.keras.utils import get_file
from deepcell import model_zoo
from deepcell.image_generators import ImageFullyConvDataGenerator
from matplotlib import pyplot as plt
from deepcell import losses
from skimage.measure import label
from skimage import morphology

In [14]:
from deepcell.utils.data_utils import get_data
from tensorflow.keras.utils import get_file

def load_data(path, mode, test_size=0.2, seed=0):
    """Loads dataset.
    Args:
         test_size (float): fraction of data to reserve as test data
        seed (int): the seed for randomly shuffling the dataset
    Returns:
           tuple: (x_train, y_train), (x_test, y_test).
    """
    #basepath = os.path.expanduser(os.path.join('~', '.keras', 'datasets'))
    #prefix = path.split(os.path.sep)[:-1]
    #data_dir = os.path.join(basepath, *prefix) if prefix else basepath
    #if not os.path.exists(data_dir):
    #    os.makedirs(data_dir)
    #elif not os.path.isdir(data_dir):
    #    raise IOError('{} exists but is not a directory'.format(data_dir))

    train_dict, test_dict = get_data(
        path,
        mode=mode,
        test_size=test_size,
        seed=seed)
    x_train, y_train = train_dict['X'], train_dict['y']
    x_test, y_test = test_dict['X'], test_dict['y']
    return (x_train, y_train), (x_test, y_test)

In [15]:
from deepcell.image_generators import ImageFullyConvDataGenerator

test_size = 0.2
seed = 0

#datagen = ImageFullyConvDataGenerator(
#    rotation_range=180,
#    zoom_range=(.8, 1.2),
#    horizontal_flip=True,
#    vertical_flip=True)

datagen = ImageFullyConvDataGenerator(
    rotation_range=30,
    zoom_range=(.9, 1.1),
    horizontal_flip=False,
    vertical_flip=False)

datagen_val = ImageFullyConvDataGenerator()

In [2]:
# Need to stack them together
## Store them into seg_validation.npz

val_all = np.load('../validation.npz')
X = val_all['X']
y = val_all['y']

In [3]:
print(X.shape)
print(y.shape)

(66, 100, 530, 530, 1)
(66, 100, 530, 530, 1)


In [4]:
X = np.vstack(X)

print(X.shape)

y = np.vstack(y)
print(y.shape)

(6600, 530, 530, 1)
(6600, 530, 530, 1)


In [8]:
X_small = X[0:-1:50]
y_small = y[0:-1:50]

In [9]:
print(X_small.shape)

(132, 530, 530, 1)


In [11]:
np.savez("../seg_validation.npz", X=X_small, y=y_small)

## Create the confusion matrix

In [12]:
# fgbg_model and pixelwise_model together
## Load the model

norm_method = 'std'  # data normalization
receptive_field = 61  # should be adjusted for the scale of the data  ## 61
n_skips = 1  # number of skip-connections (only for FC training)

dilation_radius = 1  # change dilation radius for edge dilation
separate_edge_classes = True  # break edges into cell-background edge, cell-cell edge
pixelwise_kwargs = {
    'dilation_radius': dilation_radius,
    'separate_edge_classes': separate_edge_classes,
}

input_shape=(530, 530, 1)

fgbg_model = model_zoo.bn_feature_net_skip_2D(
    n_features=2,  # segmentation mask (is_cell, is_not_cell)
    receptive_field=receptive_field,
    norm_method=norm_method,
    n_skips=n_skips,
    n_conv_filters=32,
    n_dense_filters=128,
    input_shape=input_shape,
    last_only=False)

2021-08-21 21:02:20.110507: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2021-08-21 21:02:20.133570: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2199995000 Hz
2021-08-21 21:02:20.138035: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5642e90c08e0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-08-21 21:02:20.138080: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-08-21 21:02:20.138379: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [13]:
fgbg_model.load_weights("../seg_training/attempt_1/models/test/conv_fgbg_model.h5")

In [34]:
# pixelwise_model

pixelwise_model = model_zoo.bn_feature_net_skip_2D(
    fgbg_model=fgbg_model,
    n_features=4 if separate_edge_classes else 3,
    receptive_field=receptive_field,
    norm_method=norm_method,
    n_skips=n_skips,
    n_conv_filters=32,
    n_dense_filters=128,
    last_only=False,
    input_shape=input_shape)

In [35]:
pixelwise_model.load_weights('../seg_training/attempt_1/models/test/conv_edgeseg_model.h5')

In [17]:
train_dict, test_dict = get_data('../seg_validation.npz', mode='sample', seed=0, test_size=0.95)

In [19]:
test_dict['X'].shape

(126, 530, 530, 1)

In [20]:
X_test = test_dict['X']
y_test = test_dict['y']

In [37]:
#fgbg_val_data = datagen_val.flow(
#    {'X': X_test, 'y': y_test},
#    seed=seed,
#    skip=n_skips,
#    transform='fgbg',
#    batch_size=1)

#pixelwise_val_data = datagen_val.flow(
#    {'X': X_test, 'y': y_test},
#    seed=seed,
#    skip=n_skips,
#    transform='pixelwise',
#    transform_kwargs=pixelwise_kwargs,
#    batch_size=1)

In [39]:
len(fgbg_val_data)

126

In [38]:
lst, y_true = next(fgbg_val_data)

In [59]:
val = np.load("../seg_validation.npz")
X = val['X']
print(X.shape)

(132, 530, 530, 1)


In [61]:
flag = 0

images = pixelwise_model.predict(X)[-1]
images_fgbg = fgbg_model.predict(X)[-1]

threshold = 0.8

fg_thresh = images_fgbg[..., 1] > threshold
fg_thresh = np.expand_dims(fg_thresh, axis=-1)

images_post_fgbg = images * fg_thresh

labeled_images = []
for i in range(images_post_fgbg.shape[0]):
    interior = images_post_fgbg[i, ..., 2] > .5
    labeled_image = label(interior)
    labeled_image = morphology.remove_small_objects(
        labeled_image, min_size=130, connectivity=1)  ## min_size=50
    labeled_images.append(labeled_image)
labeled_images = np.array(labeled_images)
labeled_images = np.expand_dims(labeled_images, axis=-1) 

print('labeled_images shape:', labeled_images.shape)

labeled_images shape: (132, 530, 530, 1)


In [62]:
y_true = val['y']
print(y_true.shape)

(132, 530, 530, 1)


In [78]:
# Take five images as an example
## Would expect a lot of 0s

y_pred_cm = []
y_true_cm = []

for i in range(132):
    for j in range(530):
        for k in range(530):
            if y_true[i,j,k] == 0:
                y_true_cm.append(0)
            elif y_true[i,j,k] != 0:
                y_true_cm.append(1)
            
            if labeled_images[i,j,k] == 0:
                y_pred_cm.append(0)
            elif labeled_images[i,j,k] != 0:
                y_pred_cm.append(1)
    print(i) 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131


In [79]:
y_pred_cm = np.array(y_pred_cm)

In [80]:
y_true_cm = np.array(y_true_cm)

In [81]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true_cm, y_pred_cm)
print(cm)

[[36933216    43120]
 [   40448    62016]]
